## Part 1

In [50]:
import sys
import ast
import numpy as np

def apply_convolution(data, kernel, bias):
    """
    data - InLen x InChannels
    kernel - OutChannels x InChannels x KernelSize
    bias - OutChannels

    returns OutLen x OutChannels
    """
    inLen, inChannels = data.shape
#     print('inLen', inLen, 'inChannels', inChannels)
    outChannels, _, kernelSize = kernel.shape
    outLen = inLen - kernelSize + 1
#     print('inLen:', inLen, 'kernelSize:', kernelSize, 'outLen:', outLen)
    out = np.zeros((outLen, outChannels))
    k_side = (kernelSize - 1) // 2
    for i, k in enumerate(kernel):
        b = bias[i]
#         print('k', i, 'shape:', k.shape)
        cnt = 0
        for j in range(k_side, inLen - k_side):
            cnt += 1
            chunk = data[j - k_side:j+k_side+1]
#             print("chunk shape:", chunk.shape)
            s = (chunk * k.T).sum() + b
            out[j - k_side, i] = s
#         print("# of chunks:", cnt)
    return out


In [51]:
data = np.array([[0.6685863697825855, 0.9865099796400376], [0.32297881307708, 0.9908870158650515], [0.8359169063921157, 0.5443776713017927], [0.5363888029267118, 0.34755850459471804], [0.5966372342560426, 0.9834742307894673], [0.7371274295314912, 0.03279590013405109], [0.08580648148402137, 0.2850655085982621], [0.584942134340805, 0.7981720699451806], [0.19604496972304086, 0.991819073359733], [0.5622511488322055, 0.07928952553499002], [0.24152151330089744, 0.2865696384305756], [0.882594506643994, 0.5949729821472712], [0.10820233432771786, 0.8549971123651271], [0.18754460128195194, 0.6303661925298489], [0.3551051497971416, 0.9452980688158904], [0.6525044770663634, 0.8054232618991838]])
kernel = np.array([[[0.8638059436915633, 0.4002290439648929, 0.8174398057982054, 0.34082478315585973, 0.5565832130592809], [0.08497737591188492, 0.7853885140384725, 0.1645895575029136, 0.6294907704137637, 0.8169862258229014]], [[0.21527072709338957, 0.9185427760457524, 0.5167378860756242, 0.12177789993763499, 0.4201289643444214], [0.8389450071463863, 0.6238637143288427, 0.5098771768082815, 0.1436853463091461, 0.12036561608743845]], [[0.8347050184618267, 0.12339875692133984, 0.13629086964943626, 0.623950910768403, 0.7092189295761365], [0.9578703072530402, 0.31612669923975534, 0.44018179806384916, 0.26615330385390035, 0.2745979551030847]]])
bias = np.array([0.6574440445518804, 0.3253787051567585, 0.3119672663686287])

In [52]:
target = np.array([[4.536374007375313, 3.40559782717228, 3.6420328514923983], [3.5379175544464094, 3.3156379278173618, 3.197855475391681], [3.11553432527298, 2.6461513962620473, 2.829234462423411], [3.955826212094638, 2.6848769190617205, 2.355474927521796], [3.315456250217841, 2.5537928749371677, 2.979329283784809], [3.2335971417998324, 1.8896105910662777, 2.297264030641381], [2.5503637613247276, 2.441085171765234, 2.066353981430436], [3.8006614791608166, 2.7637277539201435, 3.030330546451793], [2.8770535772488457, 2.377839924681041, 2.6997168451820808], [3.4854598947993667, 1.9636904181843315, 1.9609925225671008], [3.3707917076650484, 2.6679105220798633, 2.272378980117733], [4.179540206172764, 2.615786698404636, 3.36235456621979]])

In [53]:
apply_convolution(data, kernel, bias)

array([[4.53637401, 3.40559783, 3.64203285],
       [3.53791755, 3.31563793, 3.19785548],
       [3.11553433, 2.6461514 , 2.82923446],
       [3.95582621, 2.68487692, 2.35547493],
       [3.31545625, 2.55379287, 2.97932928],
       [3.23359714, 1.88961059, 2.29726403],
       [2.55036376, 2.44108517, 2.06635398],
       [3.80066148, 2.76372775, 3.03033055],
       [2.87705358, 2.37783992, 2.69971685],
       [3.48545989, 1.96369042, 1.96099252],
       [3.37079171, 2.66791052, 2.27237898],
       [4.17954021, 2.6157867 , 3.36235457]])

## Part 2

#### Derivative of Convolution Layer wrt parameters

In [104]:
import sys
import ast
import numpy as np

def calculate_kernel_grad(x, y, kernel, bias):
    """
    x - InLen x InChannels
    y - OutLen x OutChannels
    kernel - OutChannels x InChannels x KernelSize
    bias - OutChannels

    returns OutChannels x InChannels x KernelSize
    """
    inLen, inChannels = x.shape
    outLen, outChannels = y.shape
    _, _, ksize = kernel.shape
    dkernel = np.zeros((outChannels, inChannels, ksize))
    #print(kernel.shape, dkernel.shape)
    k_side = (ksize - 1) // 2
    for i, k in enumerate(kernel):
        b = bias[i]
#         print('k', i, 'shape:', k.shape)
        cnt = 0
        for j in range(k_side, inLen - k_side):
            cnt += 1
            chunk = x[j - k_side:j+k_side+1]
            s = (chunk * k.T).sum() + b
            #print("chunk shape:", chunk.shape)
            dkernel[i, ...] += chunk.T
#         print("# of chunks:", cnt)
    return dkernel

x = np.array([[0.1559846921787793, 0.31890243279158936, 0.4294841981370352], [0.6287087193276831, 0.041166388481120975, 0.0670771539905104], [0.15852860049868056, 0.5535509628878403, 0.7578893631796608], [0.505354018460584, 0.39104507392557886, 0.267830936523598], [0.35352084058390487, 0.09557605719492113, 0.17762289879326898], [0.17895124947325458, 0.2038143268404633, 0.45431038892117714], [0.44910520386366004, 0.28874952266426823, 0.48880576852948343], [0.978917156152191, 0.11927306276379035, 0.6831784491543058], [0.7665547409895508, 0.02661305420346527, 0.662020788170643]])
y = np.array([[0.9423069699375323, 2.235723993887441], [1.051430354504024, 2.5441990558270864], [1.3060781439427196, 2.77617352972661], [1.1097986223660692, 2.019161891632857], [0.8312656308401454, 2.1968468090151005], [1.0883546513743934, 3.182136137325698], [1.4545203460970286, 3.3294233853738975]])
kernel = np.array([[[0.7285150274194675, 0.13990616439149894, 0.08385531710791316], [0.8119118106104425, 0.19272155988045991, 0.010762309371285528], [0.5242309485683324, 0.33106798748722055, 0.2219888201243384]], [[0.31261137319823096, 0.3951341806652614, 0.954412244770657], [0.5475239344122861, 0.6407966544293683, 0.2840031545245296], [0.9267337670407934, 0.626334029479077, 0.4315268897320006]]])
bias = np.array([0.03900532471824536, 0.6619593919342232])


result = calculate_kernel_grad(x, y, kernel, bias)
print(result)

[[[2.43015332 3.25308579 3.39093181]
  [1.89280476 1.69317539 1.67862206]
  [2.64302071 2.89671496 3.49165859]]

 [[2.43015332 3.25308579 3.39093181]
  [1.89280476 1.69317539 1.67862206]
  [2.64302071 2.89671496 3.49165859]]]


## Part 3

#### Derivative wrt input

In [107]:
import sys
import ast
import numpy as np



def calculate_conv_x_grad(x, y, kernel, bias):
    """
    x - InLen x InChannels
    y - OutLen x OutChannels
    kernel - OutChannels x InChannels x KernelSize
    bias - OutChannels

    returns InLen x InChannels
    """
    inLen, inChannels = x.shape
    outLen, outChannels = y.shape
    _, _, ksize = kernel.shape
    dx = np.zeros((inLen, inChannels))
    k_side = (ksize - 1) // 2
    for i, k in enumerate(kernel):
        b = bias[i]
#         print('k', i, 'shape:', k.shape)
        cnt = 0
        for j in range(k_side, inLen - k_side):
            cnt += 1
            dx[j - k_side:j+k_side+1] += k.T
    return dx

x = np.array([[0.5031766517322117, 0.30744410216949514], [0.04690208449415345, 0.322727131626243], [0.1388690574185909, 0.48576543724022325], [0.5260018011862109, 0.5859221562109312], [0.9194272143904142, 0.3887293155713266], [0.26873714217871125, 0.9546207791313607], [0.8974007607375208, 0.5713329992292489], [0.378989716528242, 0.49787928388753266]])
y = np.array([[1.5157583762374225, 0.9460413662192456, 0.9802340338281511], [1.5728362445918327, 0.996409724139607, 1.2530013664472253], [1.9068174476481374, 1.430592927945995, 1.6704630594015581], [2.189768979209843, 2.3149543871163503, 2.1601629609824995], [2.8353457102707083, 1.7422359297539565, 1.816707087141475], [2.0532913525958474, 1.9924093441385802, 2.3069493556139014]])
kernel = np.array([[[0.8077620147648772, 0.006392942850116379, 0.6080212915877307], [0.6288229869798402, 0.6410664904844843, 0.75419330562945]], [[0.5355186530459589, 0.9211024178840701, 0.27725553497982014], [0.4507098181629161, 0.081570594016668, 0.8234980185346139]], [[0.0325944131753374, 0.7744753133142763, 0.05946983249285043], [0.7059580971549311, 0.7969953841197822, 0.5257810951530107]]])
bias = np.array([0.2579976950685653, 0.029957050945287222, 0.18958928880952108])

result = calculate_conv_x_grad(x, y, kernel, bias)
target = np.array([[1.3758750809861735, 1.7854909022976875], [3.0778457550346365, 3.305123370918622], [4.022592414095037, 5.4085957902356965], [4.022592414095037, 5.4085957902356965], [4.022592414095037, 5.4085957902356965], [4.022592414095037, 5.4085957902356965], [2.646717333108864, 3.623104887938009], [0.9447466590604012, 2.1034724193170744]])
print(result)

[[1.37587508 1.7854909 ]
 [3.07784576 3.30512337]
 [4.02259241 5.40859579]
 [4.02259241 5.40859579]
 [4.02259241 5.40859579]
 [4.02259241 5.40859579]
 [2.64671733 3.62310489]
 [0.94474666 2.10347242]]


## Part 4

#### Calculating receptive field

In [76]:
import ast
import sys
import collections
import numpy as np


LayerInfo = collections.namedtuple('LayerInfo', ('kernel_size', 'dilation'))



def calculate_receptive_field(layers):
    """
    layers - list of LayerInfo

    returns int - receptive field size
    """
    rf = 1
    for ksize, dilation in layers:
        rf += (ksize - 1) * dilation
    return rf

kernels = [9, 9, 3]
dilations = [2, 3, 4]

layers = [LayerInfo(k, d) for k, d in zip(kernels, dilations)]

result = calculate_receptive_field(layers)
print(result)

49


In [77]:
calculate_receptive_field([(3, 1), (3, 2)])

7